In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score, precision_score, recall_score
from imblearn.ensemble import BalancedRandomForestClassifier
from imblearn.pipeline import Pipeline as imbpipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
import warnings
from sklearn.exceptions import UndefinedMetricWarning
from imblearn.over_sampling import SMOTE

warnings.filterwarnings(action='ignore', category=UndefinedMetricWarning)

np.random.seed(42)

### Train test split

In [ ]:
def split(source_file):
    df = pd.read_csv(source_file)
    X = df.drop(['literature_review'], axis=1)
    y = df['literature_review']
    X_train, X_test, y_train, y_test = train_test_split(X, y , test_size=0.25, random_state=42)
    return X_train, X_test, y_train, y_test

### Parameter tuning with cross validation, smote and grid search

In [ ]:
def tune_params(X_train,y_train,pipeline,params):
    grid_search = GridSearchCV(estimator=pipeline, param_grid=params, scoring=['f1','recall','precision'],cv=5, refit='f1')
    grid_search.fit(X_train,y_train)
    return grid_search.best_params_, grid_search.score(X_train, y_train)

def cv(source_path):
    X_train, X_test, y_train, y_test = split(source_path)

    '''
    # Balanced Random Forest
    brf_pipeline = imbpipeline([('brf', BalancedRandomForestClassifier())])
    brf_params = {'brf__bootstrap': [True, False],
                'brf__max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
                'brf__max_features': ['auto', 'sqrt'],
                'brf__min_samples_leaf': [1, 2, 4],
                'brf__min_samples_split': [2, 5, 10],
                'brf__n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000],
                'brf__class_weight': [None, 'balanced']}
    brf_best_params, brf_score = tune_params(X_train, y_train, brf_pipeline, brf_params)
    print(f'Balanced Random Forest: \n best params: {brf_best_params}\n scores: {brf_score}')
    '''

    # Logistic Regression
    lr_pipeline = imbpipeline([('sampling', SMOTE()),('scaler', StandardScaler()),('lr', LogisticRegression(max_iter=200))])
    lr_params = {'lr__penalty':['l1','l2'], 
                'lr__C':[1, 10, 100, 1000],
                'lr__class_weight': [None, 'balanced']}
    lr_best_params, lr_score = tune_params(X_train, y_train, lr_pipeline, lr_params)
    print(f'Logistic Regression:\n best params: {lr_best_params}\n scores: {lr_score}')
    
    # Support Vector Machines
    svm_pipeline = imbpipeline([('sampling', SMOTE()),('scaler', StandardScaler()),('svm', SVC())])
    svm_params = {'svm__C': [0.1, 1, 10, 100, 1000],  
                'svm__gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
                'svm__kernel': ['rbf'],
                'svm__class_weight': [None, 'balanced']}
    svm_best_params, svm_score = tune_params(X_train, y_train, svm_pipeline, svm_params)
    print(f'Support Vector Machines:\n best params: {svm_best_params}\n scores: {svm_score}')

    # Naive Bayes
    f1_nb = np.mean(cross_val_score(BernoulliNB(), X_train, y_train, scoring="f1"))
    print(f"Naive Bayes: \n f1: {f1_nb} -> no parameter optimization!")

    # Decision Trees
    dt_pipeline = imbpipeline([('sampling', SMOTE()),('dt', DecisionTreeClassifier())])
    dt_params = {'dt__criterion': ['gini', 'entropy'], 
                'dt__max_depth':range(1,10),
                'dt__class_weight': [None, 'balanced']}
    dt_best_params, dt_score = tune_params(X_train, y_train, dt_pipeline, dt_params)
    print(f'Decision Trees:\n best params: {dt_best_params}\n scores: {dt_score}')

    # Random Forest
    rf_pipeline = imbpipeline([('sampling', SMOTE()),('rf', RandomForestClassifier())])
    rf_params = {'rf__bootstrap': [True, False],
                    'rf__max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
                    'rf__max_features': ['auto', 'sqrt'],
                    'rf__min_samples_leaf': [1, 2, 4],
                    'rf__min_samples_split': [2, 5, 10],
                    'rf__n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000],
                    'rf__class_weight': [None, 'balanced']}
    rf_best_params, rf_score = tune_params(X_train, y_train, rf_pipeline, rf_params)
    print(f'Random Forest:\n best params: {rf_best_params}\n scores: {rf_score}')

    # k-nearest neighbor
    knn_pipeline = imbpipeline([('sampling', SMOTE()),('scaler', StandardScaler()),('knn', KNeighborsClassifier())])
    knn_params = {'knn__n_neighbors': range(1,10),  
                'knn__weights': ['uniform', 'distance']}
    knn_best_params, knn_score = tune_params(X_train, y_train, knn_pipeline, knn_params)
    print(f'K-nearest neighbor:\n best params: {knn_best_params}\n scores: {knn_score}')

    return y_test, X_test, y_train, X_train


In [ ]:
print("- - - original dataset, keywords feature - - -")
y_test1, X_test1, y_train1, X_train1 = cv("../../data/processed/original_dataset/data_key.csv")

print("- - - original dataset, references feature - - -")
y_test2, X_test2, y_train2, X_train2 = cv("../../data/processed/original_dataset/data_ref.csv")

print("- - - original dataset, text mining feature - - -")
y_test3, X_test3, y_train3, X_train3 = cv("../../data/processed/original_dataset/data_tm.csv")

print("- - - extended dataset, keywords feature - - -")
y_test4, X_test4, y_train4, X_train4 = cv("../../data/processed/extended_dataset/data_key.csv")

print("- - - extended dataset, references feature - - -")
y_test5, X_test5, y_train5, X_train5 = cv("../../data/processed/extended_dataset/data_ref.csv")

print("- - - extended dataset, text mining feature - - -")
y_test6, X_test6, y_train6, X_train6 = cv("../../data/processed/extended_dataset/data_tm.csv")

### Training and testing with optimal parameters

In [ ]:
def train_test(estimator, y_test, X_test, y_train, X_train):
    estimator.fit(X_train, y_train)
    y_pred = estimator.predict(X_test)
    f1 = f1_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    return f1, recall, precision

print("original dataset, keywords feature")
print(" LR:")
f1, recall, precision = train_test(LogisticRegression(max_iter=200, C=100, class_weight='balanced', penalty='l2'), y_test1, X_test1, y_train1, X_train1)
print(f"     f1: {f1}\n     recall: {recall}\n      precision: {precision}")
print(" SVM:")
f1, recall, precision = train_test(SVC(C=0.1, class_weight='balanced', gamma=0.001, kernel='rbf'), y_test1, X_test1, y_train1, X_train1)
print(f"     f1: {f1}\n     recall: {recall}\n      precision: {precision}")
print(" NB:")
f1, recall, precision = train_test(BernoulliNB(), y_test1, X_test1, y_train1, X_train1)
print(f"     f1: {f1}\n     recall: {recall}\n      precision: {precision}")
print(" DT:")
f1, recall, precision = train_test(DecisionTreeClassifier(class_weight='balanced', criterion='gini', max_depth=2), y_test1, X_test1, y_train1, X_train1)
print(f"     f1: {f1}\n     recall: {recall}\n      precision: {precision}")